# Plot benchmark data

**Abstract**: I have ran some benchmarking runs with svdmodel benchmark, and saved the results to a CSV file. Here, I will make some plots regarding the obtained data. 

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

bu2022_outdir = "./outdir_Bu2022/"
bu2023_outdir = "./outdir_Bu2023/"
bu2022_high_tmin_outdir = "./outdir_Bu2022_high_tmin/"
bu2023_high_tmin_outdir = "./outdir_Bu2023_high_tmin/"
outdir = "./postprocessing/"

## Preparation

Note: All the filters used in Bu2023Ye are also in Bu2022Ye (but not vice versa)

## tmin = 0.0

In [12]:
which_bu2022 = bu2022_outdir
which_bu2023 = bu2023_outdir

bu2022_files = [f for f in os.listdir(which_bu2022) if ".csv" in f]
bu2022_filters = [f.replace(".csv", "") for f in bu2022_files]

bu2023_files = [f for f in os.listdir(which_bu2023) if ".csv" in f]
bu2023_filters = [f.replace(".csv", "") for f in bu2023_files]

filters = bu2023_filters

for filt in filters:
    bu2022_data = np.loadtxt(bu2022_outdir + f"{filt}.csv")
    bu2023_data = np.loadtxt(bu2023_outdir + f"{filt}.csv")
    
    print(filt)
    print(np.max(bu2022_data))
    print(np.max(bu2023_data))
    
    # Sort the data
    bu2022_data = np.sort(bu2022_data)[::-1]
    bu2023_data = np.sort(bu2023_data)[::-1]

    # print(bu2022_data)
    # print(bu2023_data)
    
    # Make histogram
    plt.figure()
    plt.hist(bu2022_data, bins=10000, range=(0,100), histtype="step", label="Bu2022", color = "blue")
    plt.hist(bu2023_data, bins=10000, range=(0,100), histtype="step", label="Bu2023", color = "red")
    
    # plt.xlabel(r"$\chi^2 / {\rm d.o.f.}$")
    plt.ylabel("Count")
    plt.legend()
    plt.savefig(f"{outdir}/{filt}.pdf", bbox_inches="tight")
    plt.savefig(f"{outdir}/{filt}.png", bbox_inches="tight")
    plt.close()

2massj
19.864103001421526
4937.955208075831
ps1__z
15.859011547262563
5795.414991427088
ps1__g
34.92535032374995
7058.864774453165
2massks
53.011896453567026
4378.272181498411
sdssu
8.284960432816415
9084.096097600152
ps1__y
26.459685380610793
6156.958257405681
2massh
23.39725065903423
6052.88471893461
ps1__r
15.05737571359532
5817.332886870435
ps1__i
18.22529202575738
5673.983457392165


## tmax = 0.5

In [13]:
which_bu2022 = bu2022_high_tmin_outdir
which_bu2023 = bu2023_high_tmin_outdir

bu2022_files = [f for f in os.listdir(which_bu2022) if ".csv" in f]
bu2022_filters = [f.replace(".csv", "") for f in bu2022_files]

bu2023_files = [f for f in os.listdir(which_bu2023) if ".csv" in f]
bu2023_filters = [f.replace(".csv", "") for f in bu2023_files]

filters = bu2023_filters

for filt in filters:
    bu2022_data = np.loadtxt(bu2022_outdir + f"{filt}.csv")
    bu2023_data = np.loadtxt(bu2023_outdir + f"{filt}.csv")
    
    print(filt)
    print(np.max(bu2022_data))
    print(np.max(bu2023_data))
    
    # Sort the data
    bu2022_data = np.sort(bu2022_data)[::-1]
    bu2023_data = np.sort(bu2023_data)[::-1]

    # print(bu2022_data)
    # print(bu2023_data)
    
    # Make histogram
    plt.figure()
    plt.hist(bu2022_data, bins=10000, range=(0,100), histtype="step", label="Bu2022", color = "blue")
    plt.hist(bu2023_data, bins=10000, range=(0,100), histtype="step", label="Bu2023", color = "red")
    
    # plt.xlabel(r"$\chi^2 / {\rm d.o.f.}$")
    plt.ylabel("Count")
    plt.legend()
    plt.savefig(f"{outdir}/{filt}.pdf", bbox_inches="tight")
    plt.savefig(f"{outdir}/{filt}.png", bbox_inches="tight")
    plt.close()

ps1__g
34.92535032374995
7058.864774453165
ps1__z
15.859011547262563
5795.414991427088
ps1__i
18.22529202575738
5673.983457392165
sdssu
8.284960432816415
9084.096097600152
ps1__r
15.05737571359532
5817.332886870435
2massh
23.39725065903423
6052.88471893461
ps1__y
26.459685380610793
6156.958257405681
2massks
53.011896453567026
4378.272181498411
2massj
19.864103001421526
4937.955208075831
